In [34]:
%matplotlib inline
import pyspark_csv as pycsv
import datetime as datetime
import pandas as pd
import numpy as np




prop = pd.read_csv('properati-AR-2017-08-01-properties-sell.csv', low_memory=False)
#prop.groupby('price')
prop.head(1)

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,2012-09-19,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,93.0,360.36036,1290.322581,NaN,NaN,NaN,http://www.properati.com.ar/udh_venta_casa_mer...,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,https://thumbs4.properati.com/1/ufoyqwscVk8n3e...


In [130]:
import pyspark

lista=prop['description'].tolist()
rdd1 = sc.parallelize(lista,8)
rdd1.count()

187482

In [127]:
import re,string
def removePunctuation(text):
    """Removes punctuation, changes to lowercase, and strips leading and trailing spaces.
    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated. (e.g. it's becomes its)
    Args:
        text (str): A string.
    Returns:
        str: The cleaned up string.
    """
    #exclude = set(string.punctuation)
    #text = ''.join(ch for ch in text if ch not in exclude)
    text=text.lower().strip(' ')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text
print removePunctuation('Hi, you!?    \n\tmmm')

rdd1 = rdd1.map(removePunctuation)
rdd1.take(2)

hi you    mmm


['casa con local en venta en padua2 dormitorios cocina comedor lavadero jardn    j attaguile propiedades',
 'chalet venta 4 ambientes paduaexcelente chalet a 2 cuadras de la estacion de padua 3 dormitorios con placard amplio livingcomedor cocina lavadero altillo amplio parque piscina cochera varios autos quincho con bao todos los servicios    j attaguile propiedades']

In [128]:
word = rdd1.flatMap(lambda x:x.split())
word = word.filter(lambda x: len(x)>3)
word.take(10)

['casa',
 'local',
 'venta',
 'padua2',
 'dormitorios',
 'cocina',
 'comedor',
 'lavadero',
 'jardn',
 'attaguile']

In [129]:
def text2bigrams(s):
    r = []
    l = s.split()
    for i in range(0,len(l)-1):
        r.append((l[i],l[i+1]))
    return r
print text2bigrams('hola que tal a todos')
bigramsRDD = word.flatMap(text2bigrams)
print bigramsRDD.take(10)
bigramsFreq = bigramsRDD.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
bigramsFreq.take(2)

[('hola', 'que'), ('que', 'tal'), ('tal', 'a'), ('a', 'todos')]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 75.0 failed 1 times, most recent failure: Lost task 0.0 in stage 75.0 (TID 105, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/cc/Descargas/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/cc/Descargas/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/cc/Descargas/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/cc/Descargas/spark-2.2.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipyth

In [119]:
word=word.reduceByKey(lambda a,b: a+b)
word[1]

TypeError: 'PipelinedRDD' object does not support indexing